### Querying Athena and Glue through AWS Wrangler

In [1]:
#import below libraries
import awswrangler as wr
import pandas as pd
import boto3
import warnings 
warnings.filterwarnings('ignore')
import configparser

In [2]:
#reading the credentials securely.
credents = configparser.ConfigParser()

In [3]:
#use read_file method
credents.read_file(open('credentials.config'))

In [4]:
#Reading in the credentials into Python variables. No can see them
aws_key = credents["AWS"]["KEY"]
aws_secret = credents["AWS"]["SECRET"]
region = credents["AWS"]["REGION"]

In [5]:
#Creating the Session
your_session = boto3.Session(aws_access_key_id=aws_key,
                            aws_secret_access_key=aws_secret,
                            region_name=region)

In [6]:
destination_bucket = "s3://destination-folder"

### Writing pandas Dataframe to S3

In [7]:
earners_csv = pd.read_csv("source_folder/top_earners_list.csv")
earners_csv

,id,name,E-mail,Salary,occupation
0,1,Joel,Joel@Athena.com,187069,Mathematician
1,2,Afro,Afro@Glue.aws.in,752689,Physicist
2,3,Beatles,beatles@lambda.com,975682,Algorist
3,4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


In [8]:
earners_jets = pd.read_csv("source_folder/earners_jets.csv")
earners_jets

,id,personal_jet,distance_covered
0,1,Jet 777x,589865
1,2,Global 7500,7987856
2,3,Falcon 8X,5125768
3,4,Phenom 300,312687


## Create new database

In [9]:
wr.catalog.create_database(name='learning_db',
                           boto3_session=your_session)

In [10]:
# Create the new database which will have no tables now
wr.catalog.tables(database='learning_db',\
                  boto3_session=your_session)

,Database,Table,Description,TableType,Columns,Partitions


#### Writing the above 2 dataframes into the Glue Catalog

In [11]:
wr.s3.to_csv(df=earners_csv,
        path=destination_bucket + '/top_earners',
             database='learning_db',table='top_earners',
             dataset=True,
    boto3_session=your_session)

{'paths': ['s3://destination-folder/csv/54579cdbd6ff4760b51509e1000e24bf.csv'],
 'partitions_values': {}}

In [12]:
wr.s3.to_csv(df=earners_jets,
        path=destination_bucket + '/csv',
             database='learning_db',table='their_jets',
             dataset=True,
    boto3_session=your_session)

{'paths': ['s3://destination-folder/csv/29c8b74af64b431bbd322b3e07b1ae51.csv'],
 'partitions_values': {}}

In [13]:
# Verify the data is written using list_objects

wr.s3.list_objects(destination_bucket,boto3_session=your_session)

['s3://destination-folder/csv/29c8b74af64b431bbd322b3e07b1ae51.csv',
 's3://destination-folder/csv/54579cdbd6ff4760b51509e1000e24bf.csv']

### Bringing in Athena 

## Very important : These queries are chargeable by AWS.

1) Describing the above tables

In [16]:
wr.athena.describe_table(database='learning_db',table='their_jets',
                         boto3_session=your_session)

,Column Name,Type,Partition,Comment
0,__index_level_0__,bigint,False,
1,id,bigint,False,
2,_personal_jet,string,False,
3,_distance_covered,bigint,False,


In [17]:
wr.athena.describe_table(database='learning_db',table='top_earners',
                         boto3_session=your_session)

,Column Name,Type,Partition,Comment
0,__index_level_0__,bigint,False,
1,id,bigint,False,
2,name,string,False,
3,e_mail,string,False,
4,salary,bigint,False,
5,occupation,string,False,


In [21]:
list_execution = wr.athena.list_query_executions(boto3_session=your_session)

In [23]:
wr.athena.show_create_table(table='their_jets',
                            database='learning_db',
                           boto3_session=your_session)

"CREATE EXTERNAL TABLE `their_jets`( `__index_level_0__` bigint, `id` bigint, `_personal_jet` string, `_distance_covered` bigint) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' WITH SERDEPROPERTIES ( 'escape.delim'='\\\\') STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat' LOCATION 's3://destination-folder/csv' TBLPROPERTIES ( 'areColumnsQuoted'='false', 'classification'='csv', 'columnsOrdered'='true', 'compressionType'='none', 'delimiter'=',', 'projection.enabled'='false', 'typeOfData'='file')"

In [25]:
wr.athena.read_sql_query(sql="SELECT * FROM top_earners",
                        database='learning_db',
                         boto3_session=your_session)

,__index_level_0__,id,name,e_mail,salary,occupation
0,0,1,Jet 777x,589865,<NA>,<NA>
1,1,2,Global 7500,7987856,<NA>,<NA>
2,2,3,Falcon 8X,5125768,<NA>,<NA>
3,3,4,Phenom 300,312687,<NA>,<NA>
4,0,1,Joel,Joel@Athena.com,187069,Mathematician
5,1,2,Afro,Afro@Glue.aws.in,752689,Physicist
6,2,3,Beatles,beatles@lambda.com,975682,Algorist
7,3,4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


#### Above issue has occured due to the path where the tables are....

In [28]:
#Lets delete the tables created and recreate them
wr.catalog.delete_table_if_exists(table='top_earners',
                                  database='learning_db',
                           boto3_session=your_session)

True

In [29]:
#Lets delete the tables created and recreate them
wr.catalog.delete_table_if_exists(table='their_jets',
                                  database='learning_db',
                           boto3_session=your_session)

True

In [30]:
wr.s3.to_csv(df=earners_csv,
        path=destination_bucket + '/earners',
             database='learning_db',table='top_earners',
             dataset=True,
    boto3_session=your_session)

{'paths': ['s3://destination-folder/earners/e62bc8ed300843cf9aaf3d3b9c25477e.csv'],
 'partitions_values': {}}

In [31]:
wr.s3.to_csv(df=earners_jets,
        path=destination_bucket + '/jets',
             database='learning_db',table='their_jets',
             dataset=True,
    boto3_session=your_session)

{'paths': ['s3://destination-folder/jets/2272b2b4632b45ecb7273e08c580d1c0.csv'],
 'partitions_values': {}}

In [32]:
wr.athena.read_sql_query(sql="SELECT * FROM top_earners",
                        database='learning_db',
                         boto3_session=your_session)

,__index_level_0__,id,name,e_mail,salary,occupation
0,0,1,Joel,Joel@Athena.com,187069,Mathematician
1,1,2,Afro,Afro@Glue.aws.in,752689,Physicist
2,2,3,Beatles,beatles@lambda.com,975682,Algorist
3,3,4,Snoop Dog,snoopy@apigateway.com,752689,Artificial Rapper


In [33]:
wr.athena.read_sql_query(sql="SELECT * FROM their_jets",
                        database='learning_db',
                         boto3_session=your_session)

,__index_level_0__,id,_personal_jet,_distance_covered
0,0,1,Jet 777x,589865
1,1,2,Global 7500,7987856
2,2,3,Falcon 8X,5125768
3,3,4,Phenom 300,312687


In [57]:
wr.athena.create_ctas_table(sql="""SELECT t.*,j.*
                            FROM their_jets as j
                            JOIN top_earners as t 
                            ON j.id = t.id""",
                        ctas_table='jets_earners',
                        ctas_database='learning_db',
                        s3_output=destination_bucket+'/joined_table',
                         boto3_session=your_session)

{'ctas_database': 'learning_db',
 'ctas_table': 'jets_earners',
 'ctas_query_id': '8e1b8176-4c64-4b4e-8a04-143c49948079'}

In [58]:
wr.catalog.tables(database='learning_db',
                  boto3_session=your_session)

,Database,Table,Description,TableType,Columns,Partitions
0,learning_db,their_jets,,EXTERNAL_TABLE,"__index_level_0__, id, _personal_jet, _distanc...",
1,learning_db,top_earners,,EXTERNAL_TABLE,"__index_level_0__, id, name, e_mail, salary, o...",


In [54]:
wr.catalog.databases(boto3_session=your_session)

,Database,Description
0,airbnbtables,
1,default,Default Hive database
2,learning_db,
3,localhivedb,
4,rdsdatabase,
5,tabmcqhoc,
6,youtube_data,


In [10]:
joined_df = wr.athena.read_sql_query(sql="""SELECT t.id, t.e_mail,
                            salary,occupation,j._distance_covered, 
                            j._personal_jet
                            FROM their_jets as j
                            JOIN top_earners as t 
                            ON j.id = t.id""",
                        database='learning_db',
                        s3_output=destination_bucket+'/joined_table',
                         boto3_session=your_session)

In [11]:
joined_df

,id,e_mail,salary,occupation,_distance_covered,_personal_jet
0,1,Joel@Athena.com,187069,Mathematician,589865,Jet 777x
1,2,Afro@Glue.aws.in,752689,Physicist,7987856,Global 7500
2,3,beatles@lambda.com,975682,Algorist,5125768,Falcon 8X
3,4,snoopy@apigateway.com,752689,Artificial Rapper,312687,Phenom 300


In [12]:
wr.s3.delete_objects(path='s3://aws-athena-query-results-642924624251-us-east-1/',
                    boto3_session=your_session)

In [14]:
wr.s3.list_objects(path=destination_bucket,
                   boto3_session=your_session)

['s3://destination-folder/csv/29c8b74af64b431bbd322b3e07b1ae51.csv',
 's3://destination-folder/csv/54579cdbd6ff4760b51509e1000e24bf.csv',
 's3://destination-folder/earners/e62bc8ed300843cf9aaf3d3b9c25477e.csv',
 's3://destination-folder/jets/2272b2b4632b45ecb7273e08c580d1c0.csv',
 's3://destination-folder/joined_table/tables/2635b3b8-edfd-4cff-8440-65e1ddb1302e-manifest.csv',
 's3://destination-folder/joined_table/tables/2635b3b8-edfd-4cff-8440-65e1ddb1302e.metadata',
 's3://destination-folder/joined_table/tables/5ee36faf-8c29-4131-b323-5dd2bd65d9ae-manifest.csv',
 's3://destination-folder/joined_table/tables/5ee36faf-8c29-4131-b323-5dd2bd65d9ae.metadata',
 's3://destination-folder/joined_table/temp_table_2a473b62c37f425b8407129e5036ebf7/20230220_074610_00083_sumcn_25c9f988-b564-4df2-915f-003eeb384fb6',
 's3://destination-folder/joined_table/temp_table_9f9565dd91f7460698469c3e4c96fe4b/20230220_064823_00114_x2mwu_d87f3145-031c-4ce5-ad65-a376c1649aff']

In [ ]:
### This query will not work
#wr.athena.read_sql_query(sql='SELECT * FROM jets_earners',
 #                       boto3_session=your_session,
  #                       database='learning_db',
   #                      data_source=destination_bucket+"/joined_table")

In [15]:
wr.athena.unload(sql="""SELECT t.id, t.e_mail,
                            salary,occupation,j._distance_covered, 
                            j._personal_jet
                            FROM their_jets as j
                            JOIN top_earners as t 
                            ON j.id = t.id""",
                        database='learning_db',
                        path=destination_bucket+'/unload_location',
                         boto3_session=your_session)

_QueryMetadata(execution_id='de61699d-6a18-46c6-8971-883447d5b384', dtype={'rows': 'Int64'}, parse_timestamps=[], parse_dates=[], converters={}, binaries=[], output_location='s3://destination-folder/unload_location/de61699d-6a18-46c6-8971-883447d5b384', manifest_location='s3://destination-folder/unload_location/de61699d-6a18-46c6-8971-883447d5b384-manifest.csv', raw_payload={'QueryExecutionId': 'de61699d-6a18-46c6-8971-883447d5b384', 'Query': "UNLOAD (SELECT t.id, t.e_mail,\n                            salary,occupation,j._distance_covered, \n                            j._personal_jet\n                            FROM their_jets as j\n                            JOIN top_earners as t \n                            ON j.id = t.id) TO 's3://destination-folder/unload_location' WITH (  format='PARQUET')", 'StatementType': 'DML', 'ResultConfiguration': {'OutputLocation': 's3://destination-folder/unload_location/de61699d-6a18-46c6-8971-883447d5b384'}, 'QueryExecutionContext': {'Database': 'l

#### Cleaning up the files

In [30]:
wr.s3.delete_objects(path=destination_bucket,boto3_session=your_session)

In [17]:
# Verify the data is written using list_objects

wr.s3.list_objects(destination_bucket+"/unload_location",boto3_session=your_session)

['s3://destination-folder/unload_location/20230220_075448_00032_zdr8m_f96cdc33-6a2f-4713-9d9c-fbb6358d1cf9',
 's3://destination-folder/unload_location/de61699d-6a18-46c6-8971-883447d5b384-manifest.csv',
 's3://destination-folder/unload_location/de61699d-6a18-46c6-8971-883447d5b384.metadata']

In [19]:
new_parquet = destination_bucket + '/unload_location/20230220_075448_00032_zdr8m_f96cdc33-6a2f-4713-9d9c-fbb6358d1cf9'
joined_table = wr.s3.read_parquet(path=new_parquet,boto3_session=your_session)
joined_table

,id,e_mail,salary,occupation,_distance_covered,_personal_jet
0,1,Joel@Athena.com,187069,Mathematician,589865,Jet 777x
1,2,Afro@Glue.aws.in,752689,Physicist,7987856,Global 7500
2,3,beatles@lambda.com,975682,Algorist,5125768,Falcon 8X
3,4,snoopy@apigateway.com,752689,Artificial Rapper,312687,Phenom 300
